<a href="https://colab.research.google.com/github/david-j-cox/Man-vs-Machine/blob/master/add_re24_col.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment Setup

In [2]:
# Set path to data
from google.colab import drive
drive.mount('/content/gdrive')
%cd './gdrive/My Drive/GME & MLB/'

Mounted at /content/gdrive
/content/gdrive/My Drive/GME & MLB


In [3]:
# Connect TPU
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

Tensorflow version 2.7.0
Running on TPU  ['10.78.120.194:8470']
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.78.120.194:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.78.120.194:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [4]:
# Packages we'll use
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
from google.colab import files
import zipfile, io, os
# Load in RE24 game state function; uncomment next line if first time using
# files.upload()
from RE24 import RE24_calc
from IPython.display import HTML, display, clear_output
import time
print("Packages loaded")

Packages loaded


In [42]:
# Load dfs if starting from here
data_raw = pd.read_csv('./Data/01_raw/all_pitches_08_19.csv').drop(['Unnamed: 0'], axis=1)
df_gs_spec = data_raw.copy()
df_gs_spec.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (44,88,89) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,...,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment
0,FF,2019-09-29,94.9,-2.3738,5.1379,Ian Kennedy,570731,453178,field_out,hit_into_play,NaN,NaN,NaN,NaN,3.0,Jonathan Schoop pops out to second baseman Eri...,R,R,R,KC,MIN,X,4.0,popup,1,1,2019,-0.9097,1.1972,0.3906,3.1900,NaN,NaN,NaN,2,9,Top,134.2,170.51,NaN,...,-16.6055,3.5400,1.6800,NaN,80.0,69.0,95.087,2430.0,6.245,565782,453178,664926.0,656811.0,625510.0,596144.0,600858.0,643436.0,621433.0,641531.0,54.2545,0.002,0.0,0.0,1.0,0.0,0.0,3.0,67,3,4-Seam Fastball,4,4,4,4,4,4,4,4,Standard,Standard
1,FC,2019-09-29,91.6,-2.3719,5.2549,Ian Kennedy,570731,453178,NaN,ball,NaN,NaN,NaN,NaN,14.0,NaN,R,R,R,KC,MIN,B,NaN,NaN,0,1,2019,-0.1234,0.6398,1.1979,1.7012,NaN,NaN,NaN,2,9,Top,NaN,NaN,NaN,...,-23.8739,3.5473,1.6873,NaN,NaN,NaN,92.040,2206.0,6.192,565782,453178,664926.0,656811.0,625510.0,596144.0,600858.0,643436.0,621433.0,641531.0,54.3079,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67,2,Cutter,4,4,4,4,4,4,4,4,Standard,Standard
2,FF,2019-09-29,95.0,-2.1334,5.1900,Ian Kennedy,570731,453178,NaN,called_strike,NaN,NaN,NaN,NaN,8.0,NaN,R,R,R,KC,MIN,S,NaN,NaN,0,0,2019,-0.8063,1.0940,-0.0142,1.8812,NaN,NaN,NaN,2,9,Top,NaN,NaN,NaN,...,-16.9976,3.5473,1.6873,NaN,NaN,NaN,96.259,2400.0,6.533,565782,453178,664926.0,656811.0,625510.0,596144.0,600858.0,643436.0,621433.0,641531.0,53.9663,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67,1,4-Seam Fastball,4,4,4,4,4,4,4,4,Standard,Standard
3,FF,2019-09-29,95.5,-2.3222,5.1256,Ian Kennedy,595909,453178,strikeout,swinging_strike,NaN,NaN,NaN,NaN,11.0,Jake Cave strikes out swinging.,R,L,R,KC,MIN,S,2.0,NaN,1,2,2019,-0.9011,1.0580,-0.8813,2.9723,NaN,NaN,NaN,1,9,Top,NaN,NaN,NaN,...,-18.0513,3.5400,1.6800,NaN,NaN,NaN,96.088,2564.0,6.223,565782,453178,664926.0,656811.0,625510.0,596144.0,600858.0,643436.0,621433.0,641531.0,54.2767,NaN,NaN,0.0,1.0,0.0,0.0,NaN,66,6,4-Seam Fastball,4,4,4,4,4,4,4,4,Standard,Standard
4,FF,2019-09-29,95.6,-2.1953,5.1823,Ian Kennedy,595909,453178,NaN,foul,NaN,NaN,NaN,NaN,3.0,NaN,R,L,R,KC,MIN,S,NaN,NaN,1,2,2019,-0.7817,1.2909,0.6739,3.3859,NaN,NaN,NaN,1,9,Top,NaN,NaN,NaN,...,-15.0374,3.3500,1.6300,161.0,61.0,52.2,96.121,2363.0,6.490,565782,453178,664926.0,656811.0,625510.0,596144.0,600858.0,643436.0,621433.0,641531.0,54.0092,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66,5,4-Seam Fastball,4,4,4,4,4,4,4,4,Standard,Standard


In [43]:
df = df_gs_spec[['game_year', 'pitcher', 'batter', 'pitch_type', 'game_date', 'type', 'balls', 'strikes', 
                 'on_3b',	'on_2b',	'on_1b', 	'outs_when_up', 'inning', 'inning_topbot', 'pitch_number',
                 'pitch_name',  'home_score',  'away_score', 'bat_score', 'fld_score',
                 'post_away_score', 'post_home_score', 'post_bat_score', 'post_fld_score', 
                 'events', 'description']]

In [51]:
# Save it 
for col in ['on_1b', 'on_2b', 'on_3b', 'outs_when_up']:
  df[col] = df[col].astype(float)
  df[col] = df[col].fillna(0.0)
df.to_csv('df_analysis.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [45]:
# Segment out the years into dfs
for year in df['game_year'].unique():
  temp_df = df[df['game_year']==year]
  temp_df.reset_index(drop=True)
  temp_df.to_csv(f'./Data/02_intermediate/df_{year}.csv')

# Add RE24 to each year df

In [52]:
# Function to add re24 vals to dataframes
def calc_re_24(df, year):
  re24_vals = []
  for i in range(len(df)):
    outs = df['outs_when_up'][i]
    first = df['on_1b'][i]
    second = df['on_2b'][i]
    third = df['on_3b'][i]
    val = RE24_calc(outs=outs, runner_first=first, 
                    runner_second=second, runner_third=third)
    re24_vals.append(val)
    if i%100==0:
      clear_output()
      print(f'Season: {year}\n{i} of {len(df)} completed ({int((i/len(df))*100)}%)')
  df['re_24'] = re24_vals
  return df

In [76]:
# Add RE24 to all year dfs
for year in [2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]:
  temp_df = pd.read_csv(f'./Data/02_intermediate/df_{year}.csv')
  drop_list = [i for i in list(temp_df) if 'Unnamed' in i]
  temp_df = temp_df.drop(drop_list, axis=1)
  for col in ['on_1b', 'on_2b', 'on_3b']:
    temp_df[col] = temp_df[col].fillna(0)
  temp_df = calc_re_24(temp_df, year)
  temp_df.to_csv(f'./Data/02_intermediate/df_{year}.csv')

Season: 2019
732400 of 732473 completed (99%)


# Add cols for change in RE24 pre-post

In [99]:
# Function to get change in RE24 pre-post pitch
def re24_change(df, year):
  df = df.sort_values(by=['game_date', 'inning', 'pitch_number'])

  # Change in RE24 value - continuous
  temp_list = []
  for index in range(len(df)):
    if index==0:
      temp_list.append(np.nan) 
    elif df['inning'][index-1]==df['inning'][index]:
      temp_list.append(df['re_24'][index-1]-df['re_24'][index])
    else:
      temp_list.append(np.nan)
    if index%100==0:
      clear_output()
      print(f'Season: {year}\nRE24 Continuous Change: {index} of {len(df)} completed ({int((index/len(df))*100)}%)')
  df['re_24_change_raw'] = temp_list
  
  # Categorical change
  temp_list = []
  for index in range(len(df)):
    if index==0:
      temp_list.append(np.nan) 
    elif df['inning'][index-1]==df['inning'][index]:
      if df['re_24'][index-1]>df['re_24'][index]:
        temp_list.append(1)
      elif df['re_24'][index-1]<df['re_24'][index]:
        temp_list.append(-1)
      else:
        temp_list.append(0)
    else:
      temp_list.append(np.nan)
    if index%100==0:
      clear_output()
      print(f'Season: {year}\nRE24 Categorical Change: {index} of {len(df)} completed ({int((index/len(df))*100)}%)')
  df['re_24_change_cat'] = temp_list

  return df

In [100]:
# Add RE24 changes to all dfs
for year in [2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]:
  temp_df = pd.read_csv(f'./Data/02_intermediate/df_{year}.csv')
  drop_list = [i for i in list(temp_df) if 'Unnamed' in i]
  temp_df = temp_df.drop(drop_list, axis=1)
  temp_df = temp_df.reset_index(drop=True)
  for col in ['on_1b', 'on_2b', 'on_3b']:
    temp_df[col] = temp_df[col].fillna(0)
  temp_df = re24_change(temp_df, year)
  temp_df.to_csv(f'./Data/02_intermediate/df_{year}.csv')

Season: 2019
RE24 Categorical Change: 732400 of 732473 completed (99%)
